<a href="https://colab.research.google.com/github/jjieun1212/basic_project/blob/main/pytorch_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# RNN의 하이퍼파라미터 설정
n_hidden = 35      # 은닉층 크기
lr = 0.01          # 학습률
epochs = 1000      # 에폭 수

# 학습할 문자열
string = "hello pytorch. how long can a single string be?"

# 사용할 문자 집합 정의
chars = "abcdefghijklmnopqrstuvwxyz ?!.,:;01"
char_list = [c for c in chars]
n_letters = len(char_list)  # 전체 문자 개수 (입력/출력 벡터 차원 수)

# 문자열을 one-hot 인코딩된 numpy 배열로 변환
def string_to_onehot(input_string):
    start = np.zeros((0, n_letters), dtype=int)  # 빈 배열 생성 (문자 수 × n_letters 크기로 누적)
    for ch in input_string:
        zero = np.zeros(n_letters, dtype=int)  # 모든 값 0인 벡터
        idx = char_list.index(ch)              # 해당 문자 인덱스 찾기
        zero[idx] = 1                           # 해당 위치에 1 설정 (one-hot)
        start = np.vstack([start, zero])        # 행 방향으로 쌓기
    return start

# one-hot 벡터를 문자로 디코딩
def onehot_to_word(onehot_1):
    onehot = onehot_1.detach().cpu().numpy()  # 텐서를 numpy 배열로 변환
    idx = onehot.argmax()                     # 가장 큰 값의 인덱스를 문자 인덱스로
    return char_list[idx]                     # 해당 인덱스의 문자 반환

# RNN 모델 클래스 정의
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        # 입력 → 은닉층 선형 변환
        self.i2h = nn.Linear(input_size, hidden_size)
        # 이전 은닉 → 현재 은닉
        self.h2h = nn.Linear(hidden_size, hidden_size)
        # 은닉 → 출력
        self.i2o = nn.Linear(hidden_size, output_size)
        # 활성화 함수
        self.act_fn = nn.Tanh()

    # 순전파 함수 정의
    def forward(self, input, hidden):
        # 현재 입력과 이전 은닉 상태를 이용해 새로운 은닉 상태 계산
        hidden = self.act_fn(self.i2h(input) + self.h2h(hidden))
        # 은닉 상태를 출력 벡터로 변환
        output = self.i2o(hidden)
        return output, hidden

    # 은닉 상태 초기화 (0으로 시작)
    def init_hidden(self):
        return torch.zeros(1, self.hidden_size)

# 모델 인스턴스 생성
rnn = RNN(n_letters, n_hidden, n_letters)

# 손실 함수 및 옵티마이저 설정
loss_func = nn.MSELoss()
optimizer = optim.Adam(rnn.parameters(), lr=lr)

# 문자열을 one-hot 텐서로 변환
one_hot = torch.from_numpy(string_to_onehot(string)).float()

# 모델 학습
for epoch in range(epochs):
    rnn.zero_grad()                   # 기울기 초기화
    total_loss = 0                   # 에폭별 손실 누적 변수
    hidden = rnn.init_hidden()       # 은닉 상태 초기화

    # 문자열의 각 문자에 대해 다음 문자 예측
    for j in range(one_hot.size(0) - 1):
        input_ = one_hot[j].unsqueeze(0)      # 입력 벡터 (1 x n_letters)
        target = one_hot[j + 1]               # 정답 벡터

        output, hidden = rnn(input_, hidden)  # RNN 순전파
        loss = loss_func(output.view(-1), target.view(-1))  # 손실 계산
        total_loss += loss                    # 손실 누적

    total_loss.backward()  # 역전파
    optimizer.step()       # 가중치 갱신

    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {total_loss.item()}")

# 학습한 모델로 문자열 생성
start = torch.zeros(1, n_letters)                   # 시작 입력 벡터
start[0, char_list.index(".")] = 1                  # '.'에서 시작한다고 가정

with torch.no_grad():                               # 추론 단계에서는 기울기 계산 X
    hidden = rnn.init_hidden()                      # 은닉 상태 초기화
    input_ = start
    output_string = ""                              # 생성된 문자열 저장용
    for i in range(len(string)):                    # 기존 문자열 길이만큼 생성
        output, hidden = rnn(input_, hidden)        # 한 글자씩 예측
        char = onehot_to_word(output)               # 예측 벡터 → 문자
        output_string += char                       # 문자열에 추가
        input_ = output                             # 출력 → 다음 입력

print("Generated string:")
print(output_string)  # 최종 생성된 문자열 출력


Epoch 0, Loss: 2.492001533508301
Epoch 100, Loss: 0.08108144253492355
Epoch 200, Loss: 0.027850287035107613
Epoch 300, Loss: 0.014384732581675053
Epoch 400, Loss: 0.011849197559058666
Epoch 500, Loss: 0.00890377163887024
Epoch 600, Loss: 0.004499353002756834
Epoch 700, Loss: 0.003697757376357913
Epoch 800, Loss: 0.0029422976076602936
Epoch 900, Loss: 0.0029318819288164377
Generated string:
s biatgln tr n tginr e ngin  o tglr ns trbn tra
